In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import datetime

In [2]:
vox_url = 'https://www.vox.com/'
npr_url = 'https://www.npr.org/'

In [3]:
vox_res = requests.get(vox_url)
npr_res = requests.get(npr_url)

print(vox_res.status_code)
print(npr_res.status_code)

200
200


In [4]:
npr_soup = BeautifulSoup(npr_res.content, 'lxml')
vox_soup = BeautifulSoup(vox_res.content, 'lxml')

In [5]:
#whether the article was featured or not
top_story_bool = []
headline = []
blurb = []
date_scraped = []
article_date = []
topic_tag = []
story = []

In [6]:
#NPR 

#find the top featured story section
headline_section = npr_soup.find('div', {'class':'story-wrap'})
# print(headline_section)

#get only the featured story, not including the substories in the section
headline_text = npr_soup.find('div', {'class':'story-text'})
# print(headline_text)

#get headline of top story
title = headline_text.find('h3', {'class':'title'})
headline.append(title.text)

#get the teaser blurb
teaser = headline_text.find('p', {'class':'teaser'})
# print(teaser.text)
blurb.append(teaser.text)

#record date of scrape
date_scraped.append(datetime.datetime.utcnow())
#add bool that story is headline
top_story_bool = 1

In [15]:
#grab the url for the full story 
featured_url = headline_text.find('a')['href']

# print(featured_url)
featured_page = requests.get(featured_url)
print(featured_page)

# print(featured_page.status_code)
featured_soup = BeautifulSoup(featured_page.content, 'lxml')

#section, including pictures that holds the paragraph tags
featured_text_section = featured_soup.select_one('#storytext')
print(featured_text_section)
#stackoverflow code to remove imageid tag
# imageid = featured_text_section.select_one("#res") #use regex for image id
# imageid.extract()

# for p in featured_text_section.select('p'):
#     print(p.text)
    
# paragraphs = featured_text_section.find_all('p', )
# for p in paragraphs:
#     print(p)
# #     story.append(p.text)
    
# #get the time of the story
# section = featured_soup.find('div', {'id':'story_meta'})
# print(section)

# time = featured_soup.find('time')['datetime']
# print(time)

<Response [200]>
None


In [ ]:

        
        
        
        
        
        
        
#         title = articles.find('h3')
#         print(title)
#         for title in articles.find_all('h3'):
            
#             headline.append(title.text)
#         for topics in articles.find_all('h2', {'class': 'slug'}):
#             topic_tag.append(topics.text)
#         for blurbs in articles.find_all('p', {'class': 'teaser'}):
#             if blurb:
#                 blurb.append(blurbs.text)
#             else:
#                 blurb.append(None)
#         url = articles.find('a')['href']
        
        


In [ ]:
blurb